## Agents (SerpApi, LLMMathChain)

### original code [TODO]

In [1]:
# from langchain.agents import load_tools
# from langchain.agents import initialize_agent
# from langchain.llms import OpenAI
# tools = load_tools(["serpapi", "llm-math"], llm=llm)
# agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
# agent.run("Who is the current leader of Japan? What is the largest prime number that is smaller than their age?")

## Test Tools

In [2]:
# !pip install google-search-results
# !pip install numexpr

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]
serpapi_api_key = os.environ["SERPAPI_API_KEY"]

# from getpass import getpass

# if "OPENAI_API_KEY" not in os.environ:
#     os.environ["OPENAI_API_KEY"] = getpass("Enter OpenAI API key: ")
# if "SERPAPI_API_KEY" not in os.environ:
#     os.environ["SERPAPI_API_KEY"] = getpass("Enter SerpAPI key: ")

### Tool 'calculator' (math)

In [4]:
import math
import numexpr
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI

@tool
def calculator(expression: str) -> str:
    """Calculate expression using Python's numexpr library.

    Expression should be a single line mathematical expression
    that solves the problem.

    Examples:
        "37593 * 67" for "37593 times 67"
        "37593**(1/5)" for "37593^(1/5)"
    """
    local_dict = {"pi": math.pi, "e": math.e}
    return str(
        numexpr.evaluate(
            expression.strip(),
            global_dict={},  # restrict access to globals
            local_dict=local_dict,  # add common mathematical functions
        )
    )

# Initialize the OpenAI model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Bind the calculator tool to the model
llm_with_tools = llm.bind_tools(tools=[calculator], tool_choice="any")

In [5]:
# Example usage
query = "What is 551368 divided by 82?"
result = llm_with_tools.invoke(query)
print(result)

content='' additional_kwargs={'tool_calls': [{'id': 'call_5xP8U9uLOvx8VJZ2xayanXeI', 'function': {'arguments': '{"expression":"551368 / 82"}', 'name': 'calculator'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 105, 'total_tokens': 124, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-5e043fb4-db78-4c02-aba2-c49a4e553450-0' tool_calls=[{'name': 'calculator', 'args': {'expression': '551368 / 82'}, 'id': 'call_5xP8U9uLOvx8VJZ2xayanXeI', 'type': 'tool_call'}] usage_metadata={'input_tokens': 105, 'output_tokens': 19, 'total_tokens': 124, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reas

### Tool 'web_search' (SerpAPIWrapper) + 'calculator'

In [6]:
import os
from langchain_core.tools import tool
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SerpAPIWrapper

# Create SerpAPI search tool
@tool
def web_search(query: str) -> str:
    """Perform a web search using SerpAPI and return search results."""
    search = SerpAPIWrapper()
    return search.run(query)

# Create a calculator tool for mathematical operations
@tool
def calculator(expression: str) -> str:
    """Perform mathematical calculations."""
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error in calculation: {str(e)}"

# Initialize the OpenAI model with both tools
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
llm_with_tools = llm.bind_tools(
    tools=[web_search, calculator], 
    tool_choice="any"
)

# Function to demonstrate tool usage
def demonstrate_tool_calling():
    # Example 1: Web Search
    web_search_query = "What is the latest breakthrough in AI?"
    web_search_response = llm_with_tools.invoke(
        f"Please search the web and summarize: {web_search_query}"
    )
    print("Web Search Response:")
    print(web_search_response)
    
    # Check for tool calls
    if web_search_response.tool_calls:
        for tool_call in web_search_response.tool_calls:
            print("\nTool Call Details:")
            print(f"Tool Name: {tool_call['name']}")
            print(f"Arguments: {tool_call['args']}")
    
    # Example 2: Calculator
    math_query = "What is 256 multiplied by 13, then divided by 4?"
    math_response = llm_with_tools.invoke(
        f"Please calculate: {math_query}"
    )
    print("\nMath Calculation Response:")
    print(math_response)
    
    # Check for tool calls
    if math_response.tool_calls:
        for tool_call in math_response.tool_calls:
            print("\nTool Call Details:")
            print(f"Tool Name: {tool_call['name']}")
            print(f"Arguments: {tool_call['args']}")

In [7]:
# Run the demonstration
demonstrate_tool_calling()

Web Search Response:
content='' additional_kwargs={'tool_calls': [{'id': 'call_LrhjtlJ9vPpovmyqxYJTrE66', 'function': {'arguments': '{"query":"latest breakthrough in AI 2023"}', 'name': 'web_search'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 82, 'total_tokens': 103, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-f15cdda8-e5c6-4e65-a1a3-74960adc5790-0' tool_calls=[{'name': 'web_search', 'args': {'query': 'latest breakthrough in AI 2023'}, 'id': 'call_LrhjtlJ9vPpovmyqxYJTrE66', 'type': 'tool_call'}] usage_metadata={'input_tokens': 82, 'output_tokens': 21, 'total_tokens': 103, 'input_token_details': {'audio': 0, 'cache_read':

In [8]:
# Advanced: Chaining tool calls
def advanced_tool_chain():
    complex_query = "Find the current population of Japan and then multiply it by 2"
    
    # First, search for population
    population_search = llm_with_tools.invoke(
        "What is the current population of Japan?"
    )
    
    # Extract population from search results
    if population_search.tool_calls:
        search_result = web_search.invoke(
            population_search.tool_calls[0]['args']['query']
        )
        
        # Use calculator to perform multiplication
        calculation_response = llm_with_tools.invoke(
            f"Multiply the population {search_result} by 2"
        )
        
        print("\nAdvanced Tool Chain Result:")
        print(calculation_response)

In [9]:
# Run advanced example
advanced_tool_chain()


Advanced Tool Chain Result:
content='' additional_kwargs={'tool_calls': [{'id': 'call_t05vl1BaGHsdCF07hSNpP9Nl', 'function': {'arguments': '{"expression":"124.5 * 2"}', 'name': 'calculator'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 96, 'total_tokens': 116, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'tool_calls', 'logprobs': None} id='run-d902ce00-565a-4c2f-a569-39d37787878e-0' tool_calls=[{'name': 'calculator', 'args': {'expression': '124.5 * 2'}, 'id': 'call_t05vl1BaGHsdCF07hSNpP9Nl', 'type': 'tool_call'}] usage_metadata={'input_tokens': 96, 'output_tokens': 20, 'total_tokens': 116, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_detai

## TODO: Agents (SerpApi, LLMMathChain)

In [10]:
# from langchain.agents import load_tools
# from langchain.agents import initialize_agent
# from langchain.llms import OpenAI
# tools = load_tools(["serpapi", "llm-math"], llm=llm)
# agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
# agent.run("Who is the current leader of Japan? What is the largest prime number that is smaller than their age?")